# Shared States with Agents and Conditional Routing

This notebook demonstrates advanced **state management** in the Agent Framework, showing how to use **shared state** to decouple large payloads from messages, enforce **structured agent outputs** with Pydantic models, and implement **conditional routing** based on agent decisions.

## Key Concepts

### Shared State Management
- **State Persistence**: Store large objects in workflow context instead of passing through edges
- **Lightweight References**: Pass only IDs between executors, load data on demand
- **State Namespacing**: Use prefixes (`email:`) to organize related state objects
- **Async State Operations**: `set_shared_state()` and `get_shared_state()` for context sharing

### Structured Agent Outputs
- **Pydantic Models**: Define expected agent response schemas
- **Response Format Enforcement**: `response_format` parameter ensures JSON structure
- **Type Safety**: Automatic validation and parsing of agent outputs
- **Robust Parsing**: Prevents errors from unstructured or malformed responses

### Conditional Routing
- **Dynamic Branching**: Route based on runtime data (spam vs. non-spam)
- **Condition Predicates**: Functions that evaluate message content
- **Type-Safe Routing**: Conditional edges based on typed messages
- **Guard Clauses**: Executors validate they receive expected message types

## Use Case: Email Spam Detection and Reply Drafting

This workflow processes incoming emails with two possible paths:

**Path 1 - Spam Email:**
```
store_email → spam_detection_agent → to_detection_result → handle_spam ✅
```

**Path 2 - Legitimate Email:**
```
store_email → spam_detection_agent → to_detection_result → 
submit_to_email_assistant → email_assistant_agent → finalize_and_send ✅
```

## What This Example Shows

1. **State-Based Architecture**: Large email payloads stored once, referenced by ID
2. **Structured Agent Responses**: Pydantic models enforce JSON schema from agents
3. **Conditional Branching**: Workflow routes based on spam detection result
4. **Composition Patterns**: Mix agent executors with function executors
5. **Azure OpenAI Integration**: Enterprise-ready AI agent configuration
6. **Event Streaming**: Monitor workflow execution in real-time

## Setup

Import required modules and configure Azure OpenAI.

### Prerequisites:
- Azure OpenAI deployment with GPT model
- Azure CLI authentication: Run `az login` before executing
- Environment variables for Azure OpenAI endpoint and deployment
- Pydantic for structured data validation

In [ ]:
from dotenv import load_dotenv
import asyncio
import os
from dataclasses import dataclass
from typing import Any
from uuid import uuid4

from agent_framework import (
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    executor,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel
from typing_extensions import Never

load_dotenv('../../.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

print("✅ Imports complete")
print("⚠️  Make sure you've run 'az login' for Azure authentication")

## Configuration Constants

Define constants for state management:

### State Key Patterns:
- **`EMAIL_STATE_PREFIX`**: Namespace for email objects (`"email:"`)
- **`CURRENT_EMAIL_ID_KEY`**: Pointer to the active email ID (`"current_email_id"`)

### Why Namespacing?
- Prevents key collisions in shared state
- Organizes related data logically
- Enables bulk operations (e.g., clear all emails)
- Documents state structure clearly

In [ ]:
EMAIL_STATE_PREFIX = "email:"
CURRENT_EMAIL_ID_KEY = "current_email_id"

print(f"📋 State configuration:")
print(f"   - Email storage prefix: '{EMAIL_STATE_PREFIX}'")
print(f"   - Current email ID key: '{CURRENT_EMAIL_ID_KEY}'")
print(f"\n   Example email key: '{EMAIL_STATE_PREFIX}12345-abcde-67890'")

## Define Pydantic Models for Structured Outputs

Pydantic models enforce schema validation for agent responses.

### DetectionResultAgent
- **Purpose**: Schema for spam detection agent output
- **Fields**:
  - `is_spam: bool` - Whether email is spam
  - `reason: str` - Explanation for classification
- **Usage**: Passed to `response_format` parameter in agent creation

### EmailResponse
- **Purpose**: Schema for email assistant agent output
- **Fields**:
  - `response: str` - Drafted email reply
- **Usage**: Ensures consistent reply format from assistant

### Benefits:
- ✅ **Type Safety**: Compile-time type checking
- ✅ **Automatic Validation**: Pydantic validates JSON structure
- ✅ **Clear Contracts**: Documents expected agent outputs
- ✅ **Error Prevention**: Catches malformed responses early

In [ ]:
class DetectionResultAgent(BaseModel):
    """Structured output returned by the spam detection agent."""
    is_spam: bool
    reason: str


class EmailResponse(BaseModel):
    """Structured output returned by the email assistant agent."""
    response: str


print("✅ Pydantic models defined:")
print(f"   - DetectionResultAgent: {DetectionResultAgent.model_fields.keys()}")
print(f"   - EmailResponse: {EmailResponse.model_fields.keys()}")

## Define Internal Data Models

Dataclasses for internal workflow data structures.

### DetectionResult
- **Purpose**: Internal representation of spam detection with email ID
- **Fields**:
  - `is_spam: bool` - Spam classification
  - `reason: str` - Detection reasoning
  - `email_id: str` - Reference to email in shared state
- **Usage**: Enriched version of `DetectionResultAgent` for routing

### Email
- **Purpose**: Email record stored in shared state
- **Fields**:
  - `email_id: str` - Unique identifier (UUID)
  - `email_content: str` - Full email text
- **Usage**: Stored once in shared state, referenced by ID

### Why Separate Models?
- **DetectionResultAgent**: Agent's JSON output (external contract)
- **DetectionResult**: Internal workflow data (enriched with email_id)
- **Email**: State storage format (optimized for retrieval)

In [ ]:
@dataclass
class DetectionResult:
    """Internal detection result enriched with the shared state email_id for later lookups."""
    is_spam: bool
    reason: str
    email_id: str


@dataclass
class Email:
    """In memory record stored in shared state to avoid re-sending large bodies on edges."""
    email_id: str
    email_content: str


print("✅ Internal data models defined:")
print("   - DetectionResult (for conditional routing)")
print("   - Email (for shared state storage)")

## Conditional Routing Helper

Create condition predicates for workflow branching.

### `get_condition(expected_result: bool)`
- **Purpose**: Factory function that creates condition predicates
- **Parameters**: `expected_result` - True for spam path, False for non-spam path
- **Returns**: Condition function that evaluates messages

### Condition Logic:
```python
def condition(message: Any) -> bool:
    if not isinstance(message, DetectionResult):
        return True  # Allow other message types to pass
    return message.is_spam == expected_result
```

### Why This Design?
- **Type Safety**: Check message type before accessing fields
- **Fail-Safe**: Non-DetectionResult messages pass through (avoid dead ends)
- **Flexibility**: Same helper for both spam and non-spam paths
- **Clarity**: `get_condition(True)` clearly means "spam path"

### Usage:
```python
# Route spam emails to handle_spam
.add_edge(to_detection_result, handle_spam, condition=get_condition(True))

# Route non-spam emails to assistant
.add_edge(to_detection_result, submit_to_email_assistant, condition=get_condition(False))
```

In [ ]:
def get_condition(expected_result: bool):
    """Create a condition predicate for DetectionResult.is_spam.
    
    Contract:
    - If the message is not a DetectionResult, allow it to pass to avoid accidental dead ends.
    - Otherwise, return True only when is_spam matches expected_result.
    """
    def condition(message: Any) -> bool:
        if not isinstance(message, DetectionResult):
            return True
        return message.is_spam == expected_result
    
    return condition


print("✅ Conditional routing helper defined")
print("\n   Usage:")
print("   - get_condition(True)  → Routes spam emails")
print("   - get_condition(False) → Routes non-spam emails")

## Executor 1: Store Email

The **store_email** executor persists email content in shared state and triggers spam detection.

### Responsibilities:
1. Generate unique email ID (UUID)
2. Create Email object with ID and content
3. Store email in shared state with namespaced key
4. Set current email ID pointer for downstream lookups
5. Send AgentExecutorRequest to spam detection agent

### State Management:
```python
# Store email object
await ctx.set_shared_state(f"email:{uuid}", Email(...))

# Store current email ID
await ctx.set_shared_state("current_email_id", uuid)
```

### Why Store in Shared State?
- **Memory Efficiency**: Large email content stored once, not duplicated on edges
- **Decoupling**: Executors work with IDs, not full payloads
- **Flexibility**: Easy to add email history or multi-email workflows
- **Performance**: Less data transfer between executors

### Input/Output:
- **Input**: `email_text: str` - Raw email content
- **Output**: `None` (sends AgentExecutorRequest message instead)
- **Side Effect**: Stores Email in shared state

In [ ]:
@executor(id="store_email")
async def store_email(email_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Persist the raw email content in shared state and trigger spam detection.
    
    Responsibilities:
    - Generate a unique email_id (UUID) for downstream retrieval.
    - Store the Email object under a namespaced key and set the current id pointer.
    - Emit an AgentExecutorRequest asking the detector to respond.
    """
    new_email = Email(email_id=str(uuid4()), email_content=email_text)
    await ctx.set_shared_state(f"{EMAIL_STATE_PREFIX}{new_email.email_id}", new_email)
    await ctx.set_shared_state(CURRENT_EMAIL_ID_KEY, new_email.email_id)
    
    print(f"📧 Stored email with ID: {new_email.email_id[:8]}...")
    print(f"   Content preview: {email_text[:100]}...")
    
    await ctx.send_message(
        AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=new_email.email_content)], should_respond=True)
    )


print("✅ store_email executor defined")

## Executor 2: Parse Detection Result

The **to_detection_result** executor parses spam detection agent response and enriches it with email ID.

### Responsibilities:
1. Receive AgentExecutorResponse from spam detection agent
2. Validate JSON output against DetectionResultAgent schema
3. Retrieve current email ID from shared state
4. Create enriched DetectionResult with email ID
5. Send DetectionResult for conditional routing

### Structured Output Parsing:
```python
parsed = DetectionResultAgent.model_validate_json(response.agent_run_response.text)
# Pydantic validates: {"is_spam": bool, "reason": str}
```

### State Retrieval:
```python
email_id: str = await ctx.get_shared_state(CURRENT_EMAIL_ID_KEY)
```

### Why Enrich with email_id?
- Downstream executors need to load email content
- DetectionResult becomes a complete routing decision with context
- Guards against state loss between executors

### Input/Output:
- **Input**: `response: AgentExecutorResponse` - Agent's spam detection result
- **Output**: `None` (sends DetectionResult message instead)
- **Validation**: Raises if JSON doesn't match DetectionResultAgent schema

In [ ]:
@executor(id="to_detection_result")
async def to_detection_result(response: AgentExecutorResponse, ctx: WorkflowContext[DetectionResult]) -> None:
    """Parse spam detection JSON into a structured model and enrich with email_id.
    
    Steps:
    1) Validate the agent's JSON output into DetectionResultAgent.
    2) Retrieve the current email_id from shared state.
    3) Send a typed DetectionResult for conditional routing.
    """
    parsed = DetectionResultAgent.model_validate_json(response.agent_run_response.text)
    email_id: str = await ctx.get_shared_state(CURRENT_EMAIL_ID_KEY)
    
    print(f"\n🔍 Spam detection result:")
    print(f"   Is spam: {parsed.is_spam}")
    print(f"   Reason: {parsed.reason[:100]}...")
    
    await ctx.send_message(DetectionResult(is_spam=parsed.is_spam, reason=parsed.reason, email_id=email_id))


print("✅ to_detection_result executor defined")

## Executor 3: Submit to Email Assistant (Non-Spam Path)

The **submit_to_email_assistant** executor handles non-spam emails by forwarding them to the drafting agent.

### Responsibilities:
1. Receive DetectionResult from conditional routing
2. **Guard**: Verify email is NOT spam (raise if misrouted)
3. Load original email content from shared state by ID
4. Create AgentExecutorRequest for email assistant
5. Request reply draft from assistant agent

### Guard Clause:
```python
if detection.is_spam:
    raise RuntimeError("This executor should only handle non-spam messages.")
```
- **Purpose**: Catch routing errors early
- **Type Safety**: Ensures conditional routing worked correctly
- **Debugging**: Clear error message if routing fails

### State Retrieval:
```python
email: Email = await ctx.get_shared_state(f"email:{detection.email_id}")
```
- Loads full email content using ID from DetectionResult
- Demonstrates decoupled architecture: email stored once, loaded on demand

### Input/Output:
- **Input**: `detection: DetectionResult` - Must have `is_spam=False`
- **Output**: `None` (sends AgentExecutorRequest message instead)
- **Guard**: Raises RuntimeError if `is_spam=True`

In [ ]:
@executor(id="submit_to_email_assistant")
async def submit_to_email_assistant(detection: DetectionResult, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
    """Forward non spam email content to the drafting agent.
    
    Guard:
    - This path should only receive non spam. Raise if misrouted.
    """
    if detection.is_spam:
        raise RuntimeError("This executor should only handle non-spam messages.")
    
    print("\n📨 Non-spam email detected - forwarding to assistant for reply drafting...")
    
    # Load the original content by id from shared state and forward it to the assistant.
    email: Email = await ctx.get_shared_state(f"{EMAIL_STATE_PREFIX}{detection.email_id}")
    
    print(f"   Email ID: {detection.email_id[:8]}...")
    print(f"   Content loaded from shared state: {len(email.email_content)} characters")
    
    await ctx.send_message(
        AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=email.email_content)], should_respond=True)
    )


print("✅ submit_to_email_assistant executor defined")

## Executor 4: Finalize and Send (Non-Spam Path Completion)

The **finalize_and_send** executor validates the drafted reply and yields the final output.

### Responsibilities:
1. Receive AgentExecutorResponse from email assistant agent
2. Validate JSON output against EmailResponse schema
3. Extract drafted reply from structured response
4. Yield final output (workflow completion)

### Structured Output Parsing:
```python
parsed = EmailResponse.model_validate_json(response.agent_run_response.text)
# Pydantic validates: {"response": str}
```

### Yielding Output:
```python
await ctx.yield_output(f"Email sent: {parsed.response}")
```
- **Purpose**: Mark workflow completion and return result
- **Workflow Termination**: No more executors run after yield
- **Result Collection**: Output collected by `events.get_outputs()`

### Input/Output:
- **Input**: `response: AgentExecutorResponse` - Assistant's drafted reply
- **Output Type**: `Never, str` - Never returns, yields string output
- **Side Effect**: Workflow completes with final result

In [ ]:
@executor(id="finalize_and_send")
async def finalize_and_send(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Validate the drafted reply and yield the final output."""
    parsed = EmailResponse.model_validate_json(response.agent_run_response.text)
    
    print(f"\n✅ Email reply drafted successfully!")
    print(f"   Reply preview: {parsed.response[:100]}...")
    
    await ctx.yield_output(f"Email sent: {parsed.response}")


print("✅ finalize_and_send executor defined")

## Executor 5: Handle Spam (Spam Path Completion)

The **handle_spam** executor processes spam emails and yields a spam notice.

### Responsibilities:
1. Receive DetectionResult from conditional routing
2. **Guard**: Verify email IS spam (raise if misrouted)
3. Format spam notification with reason
4. Yield final output (workflow completion)

### Guard Clause:
```python
if not detection.is_spam:
    raise RuntimeError("This executor should only handle spam messages.")
```
- **Purpose**: Ensure spam path receives only spam emails
- **Type Safety**: Validate conditional routing logic
- **Fail-Fast**: Catch routing errors immediately

### Yielding Output:
```python
await ctx.yield_output(f"Email marked as spam: {detection.reason}")
```
- Includes spam detection reasoning for auditability
- Workflow terminates after spam notification
- No email reply drafted for spam

### Input/Output:
- **Input**: `detection: DetectionResult` - Must have `is_spam=True`
- **Output Type**: `Never, str` - Never returns, yields string output
- **Guard**: Raises RuntimeError if `is_spam=False`

In [ ]:
@executor(id="handle_spam")
async def handle_spam(detection: DetectionResult, ctx: WorkflowContext[Never, str]) -> None:
    """Yield output describing why the email was marked as spam."""
    if not detection.is_spam:
        raise RuntimeError("This executor should only handle spam messages.")
    
    print(f"\n🚫 SPAM DETECTED!")
    print(f"   Reason: {detection.reason}")
    print(f"   No reply will be drafted.")
    
    await ctx.yield_output(f"Email marked as spam: {detection.reason}")


print("✅ handle_spam executor defined")

## Create Azure OpenAI Agents

Configure two specialized AI agents with structured outputs.

### Spam Detection Agent
- **Instructions**: Identify spam emails and explain reasoning
- **Response Format**: `DetectionResultAgent` (Pydantic model)
- **Output**: JSON with `is_spam` (bool) and `reason` (str)
- **Behavior**: Analyzes email content for spam indicators

### Email Assistant Agent
- **Instructions**: Draft professional email replies
- **Response Format**: `EmailResponse` (Pydantic model)
- **Output**: JSON with `response` (str) containing drafted reply
- **Behavior**: Generates appropriate responses to legitimate emails

### Why `response_format` Parameter?
- **Schema Enforcement**: Agent MUST return JSON matching the model
- **Automatic Validation**: Pydantic validates structure automatically
- **Type Safety**: Downstream code can safely access fields
- **Error Prevention**: Malformed responses caught by validation

### Azure OpenAI Configuration:
- **Credential**: `AzureCliCredential()` - Uses `az login` authentication
- **Client**: `AzureOpenAIChatClient` - Manages agent lifecycle
- **Environment Variables**: Endpoint and deployment name from env

In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)

# Create spam detection agent
spam_detection_agent = chat_client.create_agent(
    instructions=(
        "You are a spam detection assistant that identifies spam emails. "
        "Always return JSON with fields is_spam (bool) and reason (string)."
    ),
    response_format=DetectionResultAgent,
    name="spam_detection_agent",
)

print("✅ Created spam_detection_agent")
print("   Output format: DetectionResultAgent {is_spam: bool, reason: str}")

# Create email assistant agent
email_assistant_agent = chat_client.create_agent(
    instructions=(
        "You are an email assistant that helps users draft responses to emails with professionalism. "
        "Return JSON with a single field 'response' containing the drafted reply."
    ),
    response_format=EmailResponse,
    name="email_assistant_agent",
)

print("✅ Created email_assistant_agent")
print("   Output format: EmailResponse {response: str}")

## Build the Workflow Graph

Construct the workflow with conditional branching based on spam detection.

### Workflow Edges:

1. **Entry Point**: `store_email`
2. **store_email → spam_detection_agent**: Submit email for analysis
3. **spam_detection_agent → to_detection_result**: Parse agent response
4. **Conditional Branch**:
   - **Non-Spam Path** (`is_spam=False`):
     - `to_detection_result → submit_to_email_assistant`
     - `submit_to_email_assistant → email_assistant_agent`
     - `email_assistant_agent → finalize_and_send` ✅ (yields reply)
   - **Spam Path** (`is_spam=True`):
     - `to_detection_result → handle_spam` ✅ (yields spam notice)

### Conditional Edge Conditions:
```python
# Route non-spam emails
.add_edge(to_detection_result, submit_to_email_assistant, condition=get_condition(False))

# Route spam emails
.add_edge(to_detection_result, handle_spam, condition=get_condition(True))
```

### Graph Visualization:
```
                   store_email
                        ↓
              spam_detection_agent
                        ↓
                to_detection_result
                   /          \
          (spam=False)      (spam=True)
                /                \
   submit_to_email_assistant   handle_spam ✅
               ↓
       email_assistant_agent
               ↓
        finalize_and_send ✅
```

In [ ]:
# Build the workflow graph with conditional edges
workflow = (
    WorkflowBuilder()
    .set_start_executor(store_email)
    .add_edge(store_email, spam_detection_agent)
    .add_edge(spam_detection_agent, to_detection_result)
    .add_edge(to_detection_result, submit_to_email_assistant, condition=get_condition(False))
    .add_edge(to_detection_result, handle_spam, condition=get_condition(True))
    .add_edge(submit_to_email_assistant, email_assistant_agent)
    .add_edge(email_assistant_agent, finalize_and_send)
    .build()
)

print("✅ Workflow graph constructed successfully!")
print("\nWorkflow structure:")
print("   store_email → spam_detection_agent → to_detection_result")
print("      ├─ (spam=False) → submit_to_email_assistant → email_assistant_agent → finalize_and_send")
print("      └─ (spam=True)  → handle_spam")

## Load Sample Email

Load email content from resources or use a default spam example.

### Resource File:
- **Path**: `python/samples/getting_started/resources/spam.txt`
- **Fallback**: Default lottery scam text if file not found

### Sample Spam Email:
```
You are a WINNER! Click here for a free lottery offer!!!
```

### Why This Example?
- Clear spam indicators (urgency, free offers, suspicious links)
- Tests spam detection agent's classification ability
- Demonstrates conditional routing to spam path

### Testing Both Paths:
- **Spam email**: Routes to `handle_spam`
- **Legitimate email**: Routes to `email_assistant_agent` → `finalize_and_send`
- Try changing the email content to test different paths!

In [ ]:
# Try to load email from resources
resources_path = os.path.join(
    os.path.dirname(os.path.dirname(os.path.realpath("__file__"))),
    "resources",
    "spam.txt",
)

if os.path.exists(resources_path):
    with open(resources_path, encoding="utf-8") as f:
        email = f.read()
    print(f"✅ Loaded email from {resources_path}")
else:
    print("⚠️  Unable to find resource file, using default text.")
    email = "You are a WINNER! Click here for a free lottery offer!!!"

print(f"\n📧 Email to process:")
print(f"   {email[:200]}..." if len(email) > 200 else f"   {email}")

print("\n💡 Try changing the email content to test different routing paths:")
print("   - Spam example: 'URGENT! You won $1M! Click now!'")
print("   - Legitimate example: 'Hi, can we schedule a meeting next week?'")

## Run the Workflow

Execute the workflow and observe conditional routing in action.

### Expected Behavior:

**For Spam Email:**
1. `store_email`: Stores email in shared state with UUID
2. `spam_detection_agent`: Analyzes and returns `{is_spam: true, reason: "..."}`
3. `to_detection_result`: Parses JSON and enriches with email ID
4. **Conditional Routing**: Routes to `handle_spam` (spam=True)
5. `handle_spam`: Yields spam notice with reason ✅

**For Legitimate Email:**
1. `store_email`: Stores email in shared state with UUID
2. `spam_detection_agent`: Analyzes and returns `{is_spam: false, reason: "..."}`
3. `to_detection_result`: Parses JSON and enriches with email ID
4. **Conditional Routing**: Routes to `submit_to_email_assistant` (spam=False)
5. `submit_to_email_assistant`: Loads email from state and forwards to assistant
6. `email_assistant_agent`: Drafts reply and returns `{response: "..."}`
7. `finalize_and_send`: Yields drafted reply ✅

### Event Streaming:
- `await workflow.run(email)` returns execution events
- Events include executor completions, messages, and outputs
- `events.get_outputs()` extracts final workflow results

### Output:
- **Spam**: `"Email marked as spam: [reason]"`
- **Non-Spam**: `"Email sent: [drafted reply]"`

In [ ]:
print("\n" + "="*60)
print("RUNNING WORKFLOW")
print("="*60 + "\n")

# Run the workflow with streaming events
events = await workflow.run(email)
outputs = events.get_outputs()

print("\n" + "="*60)
print("WORKFLOW COMPLETE")
print("="*60)

if outputs:
    print(f"\n📋 Final result:\n   {outputs[0]}")
else:
    print("\n⚠️  No outputs generated (unexpected)")

print(f"\n📊 Execution statistics:")
print(f"   - Total events: {len(events._events) if hasattr(events, '_events') else 'N/A'}")
print(f"   - Final outputs: {len(outputs)}")

## Sample Output Analysis

### Expected Output for Spam Email:

```
Final result: Email marked as spam: This email exhibits several common spam and scam 
characteristics: unrealistic claims of large cash winnings, urgent time pressure, requests 
for sensitive personal and financial information, and a demand for a processing fee. The 
sender impersonates a generic lottery commission, and the message contains a suspicious link. 
All these are typical of phishing and lottery scam emails.
```

### Key Observations:

1. **Spam Detection Accuracy**:
   - Agent correctly identifies spam indicators
   - Detailed reasoning explains the classification
   - Mentions specific red flags (urgency, suspicious links, etc.)

2. **Conditional Routing Success**:
   - Workflow routed to `handle_spam` path
   - Email assistant was NOT invoked (correct behavior)
   - Guard clauses prevented misrouting

3. **Shared State Efficiency**:
   - Email stored once in shared state
   - Only email ID passed through routing logic
   - No duplicate email content in messages

4. **Structured Output Validation**:
   - Agent returned valid JSON matching DetectionResultAgent schema
   - Pydantic validation succeeded
   - Type-safe access to `is_spam` and `reason` fields

## Key Takeaways

### Shared State Management
- ✅ **Decouple Data from Messages**: Store large payloads in shared state, pass IDs
- ✅ **Memory Efficiency**: Email content stored once, referenced multiple times
- ✅ **Flexible Architecture**: Easy to add email history or multi-email workflows
- ✅ **Async State Operations**: Non-blocking `set_shared_state()` and `get_shared_state()`
- ✅ **State Namespacing**: Prefixes (`email:`) organize related objects
- ✅ **Type-Safe State**: Store and retrieve strongly typed objects

### Structured Agent Outputs
- ✅ **Pydantic Models**: Define expected JSON schemas for agent responses
- ✅ **Response Format Enforcement**: `response_format` parameter ensures structure
- ✅ **Automatic Validation**: Pydantic validates JSON against model
- ✅ **Type Safety**: Downstream code can safely access validated fields
- ✅ **Error Prevention**: Malformed responses caught early by validation
- ✅ **Clear Contracts**: Models document agent output expectations

### Conditional Routing
- ✅ **Dynamic Branching**: Route based on runtime data (spam vs. non-spam)
- ✅ **Condition Predicates**: Functions evaluate message content for routing
- ✅ **Type-Safe Routing**: Check message types before accessing fields
- ✅ **Guard Clauses**: Executors validate expected message types
- ✅ **Fail-Safe Design**: Non-matching messages pass through to avoid dead ends
- ✅ **Multiple Paths**: Different workflows for different classification results

### Workflow Composition
- ✅ **Mixed Executors**: Combine agent executors with function executors
- ✅ **Agent Integration**: AgentExecutor for AI-powered processing
- ✅ **Function Executors**: Custom logic for routing, validation, and formatting
- ✅ **Message Transformation**: Convert between agent responses and internal types
- ✅ **WorkflowBuilder API**: Fluent interface for graph construction

### Azure OpenAI Integration
- ✅ **Enterprise Authentication**: AzureCliCredential for secure access
- ✅ **Agent Configuration**: System instructions define behavior
- ✅ **Structured Outputs**: JSON schema enforcement via `response_format`
- ✅ **Chat Client**: Manages agent lifecycle and requests
- ✅ **Environment Configuration**: Endpoint and deployment from env vars

### Event Streaming and Observability
- ✅ **Workflow Events**: `workflow.run()` returns execution events
- ✅ **Output Collection**: `events.get_outputs()` extracts results
- ✅ **Real-Time Monitoring**: Stream events as workflow executes
- ✅ **Debugging Support**: Events provide execution traces
- ✅ **Result Aggregation**: Multiple outputs can be yielded and collected

### When to Use This Pattern
- ✅ Processing large payloads with multiple processing steps
- ✅ AI-powered classification with conditional workflows
- ✅ Need structured, validated outputs from LLMs
- ✅ Multi-path workflows based on runtime decisions
- ✅ Enterprise applications requiring type safety and validation
- ✅ Workflows with complex state management requirements

### Best Practices
- 🎯 **Use Shared State for Large Data**: Avoid duplicating payloads in messages
- 🎯 **Namespace State Keys**: Prevent collisions and organize data
- 🎯 **Enforce Structured Outputs**: Use Pydantic models with `response_format`
- 🎯 **Add Guard Clauses**: Validate message types in executors
- 🎯 **Design Fail-Safe Conditions**: Allow non-matching messages to pass
- 🎯 **Document State Schema**: Comment state key patterns and types
- 🎯 **Validate Early**: Parse and validate at routing points
- 🎯 **Enrich Data Progressively**: Add context (like email_id) as workflow progresses

### Performance Optimization
- 🚀 Store frequently accessed data in shared state
- 🚀 Use UUIDs for efficient state lookups
- 🚀 Minimize message size by passing references
- 🚀 Leverage async state operations for concurrency
- 🚀 Cache agent responses when appropriate
- 🚀 Use structured outputs to avoid retry loops

### Next Steps
- Add email history tracking with multiple emails in state
- Implement more sophisticated spam detection (ML models, rules)
- Add email categorization (urgent, informational, etc.)
- Build multi-turn conversation for email clarification
- Integrate with email services (SendGrid, Office 365)
- Add A/B testing for different agent instructions
- Implement fallback paths for agent failures
- Create dashboard for workflow monitoring and analytics